In [38]:
import numpy as np
from IPython.display import Markdown, display


def print_md(md):
    display(Markdown(md))


def array_to_md(A, precision=5):
    A = np.round(A, precision)
    md = "$\\begin{pmatrix}"
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            entry = str(A[i, j])
            md += entry + "&"
        md = md[:-1] + '\\\\'  # strip last & and add \\
    md = md[:-2]  # strip last \\
    md += "\\end{pmatrix}$"
    return md

In [39]:
def LUP(A):
    """Computes and returns an LU decomposition with pivoting. The return value  
       is a tuple (L,U,P) and fulfills L*U=P*A (* is matrix-multiplication)."""
    n = len(A)
    A = A.copy()
    L = np.identity(n)
    U = np.zeros((n, n))
    P = np.identity(n)
    

    for i in range(n):
        max_val = abs(A[i][i])
        max_row = i
        for j in range(i + 1, n):
            if abs(A[j][i]) > max_val:
                max_val = abs(A[j][i])
                max_row = j
        if i != max_row:
            A[[i, max_row]] = A[[max_row, i]]
            L[[i, max_row],:i] = L[[max_row, i],:i]
            P[[i, max_row]] = P[[max_row, i]]
        for j in range(i + 1, n):
            L[j][i] = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] = A[j][k] - L[j][i] * A[i][k]
    U = A


    return L, U, P


def ForwardSubstitution(L, b):
    """Solves the linear system of equations L*x=b assuming that L is a left lower 
       triangular matrix. It returns x as column vector."""
    n = len(L)
    x = [0] * n

    for i in range(n):
        x[i] = b[i]
        for j in range(i):
            x[i] -= L[i][j] * x[j]
        x[i] /= L[i][i]
    
    return x


def BackSubstitution(U, b):
    """Solves the linear system of equations U*x=b assuming that U is a right upper 
       triangular matrix. It returns x as column vector."""
    n = len(U)
    x = [0] * n

    for i in range(n - 1, -1, -1):
        x[i] = b[i]
        for j in range(i + 1, n):
            x[i] -= U[i][j] * x[j]
        x[i] /= U[i][i]
    
    return x


def SolveLinearSystemLUP(A, b):
    """Given a square array A and a matching vector b this function solves the 
       linear system of equations A*x=b using a pivoted LU decomposition and returns 
       x."""
    L, U, P = LUP(A)
    
    y = P@b
    x = ForwardSubstitution(L, y)
    x = BackSubstitution(U, x)
    
    return x


def LeastSquares(B, c):
    """Given a matrix A and a vector b this function solves the least squares 
       problem of minimizing |A*x-b| and returns the optimal x."""
    A = B.T@B
    b = B.T@c
    
    x = SolveLinearSystemLUP(A, b)
    x = np.array(x)
    
    return x

In [40]:
# A test matrix where LU fails but LUP works fine
A = np.array([[1, 2, 6],
              [4, 8, -1],
              [2, 3, 5]], dtype=np.double)
b = np.array([[1], [2], [3]], dtype=np.double)
# Test the LUP-decomposition
L, U, P = LUP(A)
print_md("$A=$" + array_to_md(A))
print_md("$L=$" + array_to_md(L))
print_md("$U=$" + array_to_md(U))
print_md("$P=$" + array_to_md(P))
print_md("$LU=$" + array_to_md(L @ U) + "$\\stackrel{?}{=}$" + array_to_md(P @ A) + "=PA")
print("This should be near zero (LUP check): " + str(np.linalg.norm(np.dot(L, U) - np.dot(P, A))))
# Test the method for solving a system of linear equations
print("This should be near zero (SolveLinearSystemLUP check): " + str(
    np.linalg.norm(np.dot(A, SolveLinearSystemLUP(A, b)) - b)))
# Test the method for solving linear least squares
A = np.random.rand(6, 4)
b = np.random.rand(6)
print("This should be near zero (LeastSquares check): " + str(
    np.linalg.norm(LeastSquares(A, b).flat - np.linalg.lstsq(A, b, rcond=None)[0])))

$A=$$\begin{pmatrix}1.0&2.0&6.0\\4.0&8.0&-1.0\\2.0&3.0&5.0\end{pmatrix}$

$L=$$\begin{pmatrix}1.0&0.0&0.0\\0.5&1.0&0.0\\0.25&-0.0&1.0\end{pmatrix}$

$U=$$\begin{pmatrix}4.0&8.0&-1.0\\0.0&-1.0&5.5\\0.0&0.0&6.25\end{pmatrix}$

$P=$$\begin{pmatrix}0.0&1.0&0.0\\0.0&0.0&1.0\\1.0&0.0&0.0\end{pmatrix}$

$LU=$$\begin{pmatrix}4.0&8.0&-1.0\\2.0&3.0&5.0\\1.0&2.0&6.0\end{pmatrix}$$\stackrel{?}{=}$$\begin{pmatrix}4.0&8.0&-1.0\\2.0&3.0&5.0\\1.0&2.0&6.0\end{pmatrix}$=PA

This should be near zero (LUP check): 0.0
This should be near zero (SolveLinearSystemLUP check): 4.440892098500626e-16
This should be near zero (LeastSquares check): 3.1985215122904828e-15
